## Launching CARLA and connecting the client

In [1]:
import carla
import random

# Connect to the client and retrieve the world object
client = carla.Client('localhost', 2000)
world = client.get_world()

In [3]:
settings = world.get_settings()
settings.synchronous_mode = False # Enables synchronous mode
world.apply_settings(settings)

59798

In [ ]:
client.get_available_maps()

In [ ]:
world = client.load_world('Town02_Opt')

## Spawn a car at a random location

In [4]:
# Get the blueprint library and filter for the vehicle blueprints
blueprints = world.get_blueprint_library().filter('*walker.pedestrian.*')
# blueprints = world.get_blueprint_library().filter('*vehicle.audi*')

# Get the blueprint for the vehicle model you want to spawn
# In this example, we're spawning a Tesla Model 3
# blueprint = blueprints.filter('etron')[0]
blueprint = blueprints.filter('male1_v1')[0]

# Choose a spawn location
# In this example, we're spawning the vehicle at a random location
transform = random.choice(world.get_map().get_spawn_points())

transform.location = carla.Location(x=19.350212, y=137.459961, z=0.600000)

# Spawn the vehicle
vehicle = world.spawn_actor(blueprint, transform)

RuntimeError: Spawn failed because of collision at spawn position

## Browse vehicle blueprints

In [ ]:
# Iterate over the blueprints in the blueprint library
print(len(blueprints))
for blueprint in blueprints:
    # Print the ID of the vehicle
    print('ID:', blueprint.id)
    
    # # Print the attributes of the vehicle
    # for attribute in vehicle:
    #     print('Attribute:', attribute)

## Move spectator to the vehicle's location

In [9]:
# Get the spectator
spectator = world.get_spectator()

# Move the spectator to the spawn position of the vehicle
spectator.set_transform(vehicle.get_transform())

## Look at the vehicle

In [ ]:
import math

# Get the location of the vehicle
vehicle_location = vehicle.get_location()

# Set the location of the spectator
spectator_location = carla.Location(vehicle_location.x + 4, vehicle_location.y + 4, vehicle_location.z + 4)  # 10 meters above the vehicle
spectator.set_location(spectator_location)

# Calculate the direction from the spectator to the vehicle
direction = vehicle_location - spectator_location

# Calculate the rotation that makes the spectator look at the vehicle
rotation = carla.Rotation()
rotation.yaw = +math.degrees(math.atan2(direction.y, direction.x))  # Yaw angle
rotation.pitch = +math.degrees(math.atan2(direction.z, math.sqrt(direction.x**2 + direction.y**2)))  # Pitch angle

# Set the rotation of the spectator
spectator.set_transform(carla.Transform(spectator_location, rotation))

## Spectate around the spawned vehicle in a circle and look at the vehicle

In [ ]:
import math
import time

# Define the radius of the circle and the height of the spectator
radius = 10  # 10 meters
height = 5   # 5 meters

# Define the speed of the rotation (in radians per second)
speed = 0.5

# Get the current time
start_time = time.time()

while True:
    # Calculate the angle based on the elapsed time and the speed
    angle = speed * (time.time() - start_time)

    # Get the location of the vehicle
    vehicle_location = vehicle.get_location()

    # Calculate the new location of the spectator
    location = carla.Location()
    location.x = vehicle_location.x - radius * math.cos(angle)  # X-coordinate
    location.y = vehicle_location.y + radius * math.sin(angle)  # Y-coordinate
    location.z = vehicle_location.z + height                    # Z-coordinate

    # Calculate the rotation that makes the spectator look at the vehicle
    rotation = carla.Rotation()
    rotation.yaw = -math.degrees(angle)  # Yaw angle
    rotation.pitch = -math.degrees(math.atan(height / radius))  # Pitch angle

    # Create a new transform with the new location and the new rotation
    transform = carla.Transform(location, rotation)

    # Set the transform of the spectator
    spectator.set_transform(transform)

    # Wait for a short moment before the next update
    time.sleep(0.01)

## Dynamic weather

In [ ]:
"""
CARLA Dynamic Weather:

Connect to a CARLA Simulator instance and control the weather. Change Sun
position smoothly with time and generate storms occasionally.
"""

import glob
import os
import sys

try:
    sys.path.append(glob.glob('../carla/dist/carla-*%d.%d-%s.egg' % (
        sys.version_info.major,
        sys.version_info.minor,
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
except IndexError:
    pass

import carla

import argparse
import math
import time


def clamp(value, minimum=0.0, maximum=100.0):
    return max(minimum, min(value, maximum))


class Sun(object):
    def __init__(self, azimuth, altitude):
        self.azimuth = azimuth
        self.altitude = altitude
        self._t = 0.0

    def tick(self, delta_seconds):
        self._t += 0.008 * delta_seconds
        self._t %= 2.0 * math.pi
        self.azimuth += 0.25 * delta_seconds
        self.azimuth %= 360.0
        self.altitude = (70 * math.sin(self._t)) - 20

    def __str__(self):
        return 'Sun(alt: %.2f, azm: %.2f)' % (self.altitude, self.azimuth)


class Storm(object):
    def __init__(self, precipitation):
        self._t = precipitation if precipitation > 0.0 else -50.0
        self._increasing = True
        self.clouds = 0.0
        self.rain = 0.0
        self.wetness = 0.0
        self.puddles = 0.0
        self.wind = 0.0
        self.fog = 0.0

    def tick(self, delta_seconds):
        delta = (1.3 if self._increasing else -1.3) * delta_seconds
        self._t = clamp(delta + self._t, -250.0, 100.0)
        self.clouds = clamp(self._t + 40.0, 0.0, 90.0)
        self.rain = clamp(self._t, 0.0, 80.0)
        delay = -10.0 if self._increasing else 90.0
        self.puddles = clamp(self._t + delay, 0.0, 85.0)
        self.wetness = clamp(self._t * 5, 0.0, 100.0)
        self.wind = 5.0 if self.clouds <= 20 else 90 if self.clouds >= 70 else 40
        self.fog = clamp(self._t - 10, 0.0, 30.0)
        if self._t == -250.0:
            self._increasing = True
        if self._t == 100.0:
            self._increasing = False

    def __str__(self):
        return 'Storm(clouds=%d%%, rain=%d%%, wind=%d%%)' % (self.clouds, self.rain, self.wind)


class Weather(object):
    def __init__(self, weather):
        self.weather = weather
        self._sun = Sun(weather.sun_azimuth_angle, weather.sun_altitude_angle)
        self._storm = Storm(weather.precipitation)

    def tick(self, delta_seconds):
        self._sun.tick(delta_seconds)
        self._storm.tick(delta_seconds)
        self.weather.cloudiness = self._storm.clouds
        self.weather.precipitation = self._storm.rain
        self.weather.precipitation_deposits = self._storm.puddles
        self.weather.wind_intensity = self._storm.wind
        self.weather.fog_density = self._storm.fog
        self.weather.wetness = self._storm.wetness
        self.weather.sun_azimuth_angle = self._sun.azimuth
        self.weather.sun_altitude_angle = self._sun.altitude

    def __str__(self):
        return '%s %s' % (self._sun, self._storm)


speed_weather_changing = 30.0
update_freq = 0.1 / speed_weather_changing
# Define the radius of the circle and the height of the spectator
radius = 10  # 10 meters
height = 5   # 5 meters
# Define the speed of the rotation (in radians per second)
speed_rotation = 0.5
start_time = time.time()

weather = Weather(world.get_weather())

elapsed_time = 0.0

while True:
    timestamp = world.wait_for_tick(seconds=30.0).timestamp
    elapsed_time += timestamp.delta_seconds
    if elapsed_time > update_freq:
        weather.tick(speed_weather_changing * elapsed_time)
        world.set_weather(weather.weather)
        sys.stdout.write('\r' + str(weather) + 12 * ' ')
        sys.stdout.flush()
        elapsed_time = 0.0
        
        # Calculate the angle based on the elapsed time and the speed_rotation
        angle = speed_rotation * (time.time() - start_time)

        # Get the location of the vehicle
        vehicle_location = vehicle.get_location()

        # Calculate the new location of the spectator
        location = carla.Location()
        location.x = vehicle_location.x - radius * math.cos(angle)  # X-coordinate
        location.y = vehicle_location.y + radius * math.sin(angle)  # Y-coordinate
        location.z = vehicle_location.z + height                    # Z-coordinate

        # Calculate the rotation that makes the spectator look at the vehicle
        rotation = carla.Rotation()
        rotation.yaw = -math.degrees(angle)  # Yaw angle
        rotation.pitch = -math.degrees(math.atan(height / radius))  # Pitch angle

        # Create a new transform with the new location and the new rotation
        transform = carla.Transform(location, rotation)

        # Set the transform of the spectator
        spectator.set_transform(transform)


## Export images from the spectator's view in CARLA

In [ ]:
# Import the necessary libraries
import carla
import os

# Define a function to save images
def save_image(image):
    image.save_to_disk(os.path.join('./output', '%06d.png' % image.frame))

# Create a blueprint for the camera
blueprint = world.get_blueprint_library().find('sensor.camera.rgb')

# Set the resolution of the camera
blueprint.set_attribute('image_size_x', '800')
blueprint.set_attribute('image_size_y', '600')

# Attach the camera to the spectator
transform = carla.Transform(carla.Location(x=0.0, y=0.0, z=0))
camera = world.spawn_actor(blueprint, transform, attach_to=spectator)

# Register the function to save images
camera.listen(save_image)

# Wait for a few seconds to let the camera capture images
time.sleep(2)

# Stop the camera
camera.stop()

# Destroy the camera
camera.destroy()